# Building a Neural Network from Scratch

https://towardsdatascience.com/math-neural-network-from-scratch-in-python-d6da9f29ce65 

### Abstract Base Class : Layer
The abstract class Layer, which all other layers will inherit from, handles simple properties which are an input, an output, and both a forward and backward methods.

In [ ]:

# Base class
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    # computes the output Y of a layer for a given input X
    def forward_propagation(self, input):
        raise NotImplementedError

    # computes dE/dX for a given dE/dY (and update parameters if any)
    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

In the abstract class above, backward_propagation function has an extra parameter, learning_rate, which is controlling the amount of learning/updating parameters using gradient descent.

### Backward Propagation
Suppose we have a matrix containing the derivative of the error with respect to that layer’s output: $\frac{\partial E}{\partial Y}$

We need :
- The derivative of the error with respect to the parameters ($\frac{\partial E}{\partial W}$, $\frac{\partial E}{\partial B}$)
- The derivative of the error with respect to the input ($\frac{\partial E}{\partial X}$)

Let's calculate $\frac{\partial E}{\partial W}$. This matrix should be the same size as $W$ itself : 

$i x j$ where $i$ is the number of input neurons and $j$ the number of output neurons. We need one gradient for every weight

### Coding the Fully Connected Layer

In [ ]:
#from layer import Layer
import numpy as np

# inherit from base class Layer
class FCLayer(Layer):
    # input_size = number of input neurons
    # output_size = number of edges that connects to neurons in next layer
    def __init__(self, input_size, output_size):
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5

    # returns output for a given input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    # computes dE/dW, dE/dB for a given output_error=dE/dY. Returns input_error=dE/dX.
    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        # dBias = output_error

        # update parameters
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

### Activation Layer
All the calculation we did until now were completely linear, may not learn well. We need to add non-linearity to the model by applying non-linear functions to the output of some layers.

Now we need to redo the whole process for this new type of layer!

In [ ]:
#from layer import Layer

# inherit from base class Layer
class ActivationLayer(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    # returns the activated input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    # Returns input_error=dE/dX for a given output_error=dE/dY.
    # learning_rate is not used because there is no "learnable" parameters.
    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error

You can also write some activation functions and their derivatives in a separate file. These will be used later to create an ActivationLayer.

In [ ]:
import numpy as np

# activation function and its derivative
def tanh(x):
    return np.tanh(x);

def tanh_prime(x):
    return 1-np.tanh(x)**2;

def Relu(x):
  return (np.maximum(0,x))

def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def sigmoid_prime(z) :
    """ Returns the derivative of the sigmoid function. """
    return sigmoid(z)*(1-sigmoid(z))

def Relu_prime(z) :
    """ Returns the derivative of the ReLU function. """
    return 1*(z>=0)    



### Loss Function
Until now, for a given layer, we supposed that ∂E/∂Y was given (by the next layer). But what happens to the last layer? How does it get ∂E/∂Y? We simply give it manually, and it depends on how we define the error.
The error of the network, which measures how good or bad the network did for a given input data, is defined by you. 

There are many ways to define the error, and one of the most known is called MSE — Mean Squared Error.

In [ ]:

import numpy as np

# loss function and its derivative
def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2));

def mse_prime(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size;

### Network Class
Almost done ! We are going to make a Network class to create neural networks very easily using the building blocks we have prepared so far.


In [ ]:
# example of a function for calculating softmax for a list of numbers
from numpy import exp
 
# calculate the softmax of a vector
def softmax(vector):
    e = exp(vector)
    return e / e.sum()

In [ ]:
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)

    # set loss to use
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

        
    # predict output for given input
    def predict(self, input_data):
        # sample dimension first
        samples = len(input_data)
        result = []

        # run network over all samples
        for i in range(samples):
            # forward propagation
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result

    # train the network 
    
    def fit(self, x_train, y_train, epochs, learning_rate):
        '''
        Fit function does the training. 
        Training data is passed 1-by-1 through the network layers during forward propagation.
        Loss (error) is calculated for each input and back propagation is performed via partial 
        derivatives on each layer.
        '''
        # sample dimension first
        samples = len(x_train)

        # training loop
        for i in range(epochs):
            err = 0
            for j in range(samples):
                # forward propagation
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)

                # compute loss (for display purpose only)
                err += self.loss(y_train[j], output)

                # backward propagation
                error = self.loss_prime(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)

            # calculate average error on all samples
            err /= samples
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))

### Building Neural Networks
Finally ! We can use our class to create a neural network with as many layers as we want ! We are going to build two neural networks : a simple XOR and a MNIST solver.


### Solve XOR
Starting with XOR is always important as it’s a simple way to tell if the network is learning anything at all.

In [ ]:
import numpy as np

#from network import Network
#from fc_layer import FCLayer
#from activation_layer import ActivationLayer
#from activations import tanh, tanh_prime
#from losses import mse, mse_prime

# training data
x_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
y_train = np.array([[[0]], [[1]], [[1]], [[0]]])

# network
net = Network()
net.add(FCLayer(2, 3))
net.add(ActivationLayer(Relu, Relu_prime))
net.add(FCLayer(3, 1))
net.add(ActivationLayer(sigmoid, sigmoid_prime))

# train
net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=1000, learning_rate=0.1)

# test
out = net.predict(x_train)
print(out)

epoch 1/1000   error=0.264831
epoch 2/1000   error=0.263669
epoch 3/1000   error=0.262622
epoch 4/1000   error=0.261676
epoch 5/1000   error=0.260820
epoch 6/1000   error=0.260042
epoch 7/1000   error=0.259335
epoch 8/1000   error=0.258958
epoch 9/1000   error=0.258671
epoch 10/1000   error=0.258417
epoch 11/1000   error=0.258193
epoch 12/1000   error=0.257993
epoch 13/1000   error=0.257813
epoch 14/1000   error=0.257652
epoch 15/1000   error=0.257505
epoch 16/1000   error=0.257371
epoch 17/1000   error=0.257247
epoch 18/1000   error=0.257133
epoch 19/1000   error=0.257025
epoch 20/1000   error=0.256923
epoch 21/1000   error=0.256826
epoch 22/1000   error=0.256733
epoch 23/1000   error=0.256642
epoch 24/1000   error=0.256554
epoch 25/1000   error=0.256467
epoch 26/1000   error=0.256380
epoch 27/1000   error=0.256293
epoch 28/1000   error=0.256206
epoch 29/1000   error=0.256118
epoch 30/1000   error=0.256029
epoch 31/1000   error=0.255938
epoch 32/1000   error=0.255844
epoch 33/1000   e

### Solve MNIST
We didn’t implemented the Convolutional Layer but this is not a problem. 
All we need to do is to reshape our data so that it can fit into a Fully Connected Layer.
MNIST Dataset consists of images of digits from 0 to 9, of shape 28x28x1. 
The goal is to predict what digit is drawn on a picture.

In [ ]:
import numpy as np

#from network import Network
#from fc_layer import FCLayer
#from activation_layer import ActivationLayer
#from activations import tanh, tanh_prime
#from losses import mse, mse_prime

from keras.datasets import mnist
from keras.utils import np_utils

# load MNIST from server
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# training data : 60000 samples
# reshape and normalize input data
x_train = x_train.reshape(x_train.shape[0], 1, 28*28)
x_train = x_train.astype('float32')
x_train /= 255
# encode output which is a number in range [0,9] into a vector of size 10
# e.g. number 3 will become [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
y_train = np_utils.to_categorical(y_train)

# same for test data : 10000 samples
x_test = x_test.reshape(x_test.shape[0], 1, 28*28)
x_test = x_test.astype('float32')
x_test /= 255
y_test = np_utils.to_categorical(y_test)

# Network
net = Network()
net.add(FCLayer(28*28, 100))                # input_shape=(1, 28*28)    ;   output_shape=(1, 100)
net.add(ActivationLayer(Relu, Relu_prime))
net.add(FCLayer(100, 50))                   # input_shape=(1, 100)      ;   output_shape=(1, 50)
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(50, 10))                    # input_shape=(1, 50)       ;   output_shape=(1, 10)
net.add(ActivationLayer(sigmoid, sigmoid_prime))

# train on 1000 samples
# as we didn't implemented mini-batch GD, training will be pretty slow if we update at each iteration on 60000 samples...
net.use(mse, mse_prime)
net.fit(x_train[0:1000], y_train[0:1000], epochs=35, learning_rate=0.1)

# test on 3 samples
out = net.predict(x_test[0:3])
print("\n")
print("predicted values : ")
print(out, end="\n")
print("true values : ")
print(y_test[0:3])

epoch 1/35   error=0.092121
epoch 2/35   error=0.061167
epoch 3/35   error=0.047518
epoch 4/35   error=0.038354
epoch 5/35   error=0.030949
epoch 6/35   error=0.025401
epoch 7/35   error=0.021587
epoch 8/35   error=0.018724
epoch 9/35   error=0.016503
epoch 10/35   error=0.014740
epoch 11/35   error=0.013220
epoch 12/35   error=0.012004
epoch 13/35   error=0.010997
epoch 14/35   error=0.010145
epoch 15/35   error=0.009381
epoch 16/35   error=0.008662
epoch 17/35   error=0.008055
epoch 18/35   error=0.007537
epoch 19/35   error=0.007098
epoch 20/35   error=0.006676
epoch 21/35   error=0.006219
epoch 22/35   error=0.005779
epoch 23/35   error=0.005409
epoch 24/35   error=0.005108
epoch 25/35   error=0.004854
epoch 26/35   error=0.004617
epoch 27/35   error=0.004406
epoch 28/35   error=0.004210
epoch 29/35   error=0.004028
epoch 30/35   error=0.003874
epoch 31/35   error=0.003743
epoch 32/35   error=0.003622
epoch 33/35   error=0.003505
epoch 34/35   error=0.003400
epoch 35/35   error=0.0

In [ ]:
#*************** deneme

[[[0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]]]


In [ ]:
import tensorflow as tf

In [ ]:
# Create a model object
model = tf.keras.Sequential()

In [ ]:
# Create an input layer
input_layer = tf.keras.layers.Dense(4096, input_shape=(34,), activation = 'relu')

# Add input layer to model object
model.add(input_layer)

In [ ]:
# Add the first hidden layer with 4096 nodes and relu activation function
model.add(tf.keras.layers.Dense(4096,activation='relu' ))
# Add 0.5 dropout
model.add(tf.keras.layers.Dropout(0.5))

# Add the second hidden layer with 4096 nodes and relu activation function
model.add(tf.keras.layers.Dense(4096,activation= 'relu'))
# Add 0.5 dropout
model.add(tf.keras.layers.Dropout(0.5))

# Add the third hidden layer with 4096 nodes and relu activation function
model.add(tf.keras.layers.Dense(4096, activation = 'relu'))
# Add 0.5 dropout
model.add(tf.keras.layers.Dropout(0.5))

# Add the fourth hidden layer with 4096 nodes and relu activation function
model.add(tf.keras.layers.Dense(4096, activation = 'relu'))
# Add 0.5 dropout
model.add(tf.keras.layers.Dropout(0.5))

In [ ]:
# Add the output layer
model.add((tf.keras.layers.Dense(7,activation= 'sigmoid')))

In [ ]:
# Compile the model
model.compile(optimizer= 'adam',loss='sparse_categorical_crosssentropy', metrics='accuracy')

In [ ]:
def Relu(x):
  return (np.maximum(0,x))

In [ ]:
Relu(out)

array([[[0.00000000e+00, 0.00000000e+00, 9.13950635e-03, 0.00000000e+00,
         0.00000000e+00, 3.31557664e-02, 1.24326872e-03, 9.46212371e-01,
         4.61049064e-02, 0.00000000e+00]],

       [[6.39176654e-01, 8.59287947e-04, 3.50408496e-01, 2.05577367e-01,
         3.50042524e-04, 1.46916474e-01, 4.99143692e-02, 0.00000000e+00,
         2.02294101e-01, 0.00000000e+00]],

       [[0.00000000e+00, 9.69141410e-01, 0.00000000e+00, 5.86614302e-04,
         0.00000000e+00, 3.58262614e-02, 9.31574587e-03, 2.16806137e-02,
         2.24608761e-02, 0.00000000e+00]]])

In [ ]:
Relu(y_test)

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
Relu(net.fit(x_train[0:1000], y_train[0:1000], epochs=30, learning_rate=0.4))

epoch 1/30   error=0.389830
epoch 2/30   error=0.388893
epoch 3/30   error=0.355355
epoch 4/30   error=0.313085
epoch 5/30   error=0.313995
epoch 6/30   error=0.279213
epoch 7/30   error=0.276015
epoch 8/30   error=0.274463
epoch 9/30   error=0.265774
epoch 10/30   error=0.262936
epoch 11/30   error=0.263309
epoch 12/30   error=0.262655
epoch 13/30   error=0.262807
epoch 14/30   error=0.258354
epoch 15/30   error=0.260744
epoch 16/30   error=0.250679
epoch 17/30   error=0.252270
epoch 18/30   error=0.253737
epoch 19/30   error=0.245494
epoch 20/30   error=0.243576
epoch 21/30   error=0.244558
epoch 22/30   error=0.246718
epoch 23/30   error=0.252411
epoch 24/30   error=0.236109
epoch 25/30   error=0.236762
epoch 26/30   error=0.243511
epoch 27/30   error=0.243719
epoch 28/30   error=0.233495
epoch 29/30   error=0.224605
epoch 30/30   error=0.224174


TypeError: ignored

In [ ]:
import numpy as np

def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

print(sigmoid(y_test))

[[0.5       0.5       0.5       ... 0.7310586 0.5       0.5      ]
 [0.5       0.5       0.7310586 ... 0.5       0.5       0.5      ]
 [0.5       0.7310586 0.5       ... 0.5       0.5       0.5      ]
 ...
 [0.5       0.5       0.5       ... 0.5       0.5       0.5      ]
 [0.5       0.5       0.5       ... 0.5       0.5       0.5      ]
 [0.5       0.5       0.5       ... 0.5       0.5       0.5      ]]
